# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

[/home/runner/work/polars/polars/polars/polars-io/src/ndjson/core.rs:162] &data_type = Struct(
    [
        Field {
            name: "Context_Name",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_MaritalStatus",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_CLV",
            data_type: Int64,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_City",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "IH_Web_Inbound_Accepted_pxLastGroupID",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Decision_Outcome",
            data_type: LargeUtf8,
            is_null

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""46206085420593…","""../../../../da…","""83246850668378…","""80165505440841…","""11704961701643…",false
1.4856e19,"""16427630891846…","""../../../../da…","""42219822631184…","""80165505440841…","""14106914793142…",true
5.6458e17,"""72047525066269…","""../../../../da…","""18163512100511…","""32501601091527…","""11704961701643…",false
4.0723e18,"""14161315516668…","""../../../../da…","""42219822631184…","""83862443376292…","""11704961701643…",false
1.4677e19,"""17747620851370…","""../../../../da…","""83246850668378…","""83862443376292…","""14054257564665…",true
1.6330e19,"""95709609765117…","""../../../../da…","""18163512100511…","""54609335890377…",null,false
8.7675e18,"""11182138909439…","""../../../../da…","""18163512100511…","""83862443376292…","""14054257564665…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""84790836205166…","""../../../../da…","""15778270697403…","""FirstMortgage3…","""34580056492919…","""Rejected"""
1.4856e19,"""15839270322715…","""../../../../da…","""58968768912145…","""FirstMortgage3…","""18388685828810…","""Accepted"""
5.6458e17,"""17470896009045…","""../../../../da…","""49311643565112…","""MoneyMarketSav…","""34580056492919…","""Rejected"""
4.0723e18,"""12279990710405…","""../../../../da…","""58968768912145…","""BasicChecking""","""34580056492919…","""Rejected"""
1.4677e19,"""45230955963394…","""../../../../da…","""15778270697403…","""BasicChecking""","""59884713285636…","""Accepted"""
1.6330e19,"""16862601823104…","""../../../../da…","""49311643565112…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""17633472480430…","""../../../../da…","""49311643565112…","""BasicChecking""","""59884713285636…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""51585139116642…","""../../../../da…","""10567941482173…","""FirstMortgage3…","""14276371814744…","""Rejected"""
1.4856e19,"""89077430554625…","""../../../../da…","""14390542144113…","""FirstMortgage3…","""18537491943467…","""Accepted"""
5.6458e17,"""67753411876396…","""../../../../da…","""17333566460845…","""MoneyMarketSav…","""14276371814744…","""Rejected"""
4.0723e18,"""15786683514798…","""../../../../da…","""14390542144113…","""BasicChecking""","""14276371814744…","""Rejected"""
1.4677e19,"""86746780287612…","""../../../../da…","""10567941482173…","""BasicChecking""","""14331668129927…","""Accepted"""
1.6330e19,"""12953503836572…","""../../../../da…","""17333566460845…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""13637196850384…","""../../../../da…","""17333566460845…","""BasicChecking""","""14331668129927…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""16540965353018…","""54548307341303…","""FirstMortgage3…","""14897328882073…","""Rejected"""
1.4856e19,"""39017025197016…","""14591791407318…","""FirstMortgage3…","""15180704685296…","""Accepted"""
5.6458e17,"""16293977266018…","""98265869572779…","""MoneyMarketSav…","""14897328882073…","""Rejected"""
4.0723e18,"""12804477109685…","""14591791407318…","""BasicChecking""","""14897328882073…","""Rejected"""
1.4677e19,"""30420655484401…","""54548307341303…","""BasicChecking""","""13745343325158…","""Accepted"""
1.6330e19,"""65722634504567…","""98265869572779…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""11224569503691…","""98265869572779…","""BasicChecking""","""13745343325158…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'